In [2]:
import sys
sys.path.append('..')

import shutil
from pathlib import Path
import re

In [ ]:
def parse_filename(filename):
    """
    Parse PIRATE calibration filename and extract relevant information.
    
    Parameters
    ----------
    filename : str
        Original PIRATE filename
        
    Returns
    -------
    dict or None
        Dictionary with parsed information or None if parsing fails
    """
    # Remove .fits extension
    name = filename.replace('.fits', '')
    
    # Split by underscores
    parts = name.split('_')
    
    # Determine if it's a flat or bias
    if 'flats' in name.lower():
        frame_type = 'flat'
        # Format: PIRATE_NUMBER_flats_FILTER_ID_YYYY_MM_DD_HH_MM_SS
        try:
            filter_band = parts[3]  # B, V, R, etc.
            frame_id = parts[4]
            date_parts = parts[5:8]  # YYYY, MM, DD
            date = '_'.join(date_parts)
            
            return {
                'type': frame_type,
                'filter': filter_band,
                'date': date,
                'id': frame_id,
                'original': filename
            }
        except IndexError:
            print(f"Could not parse flat file: {filename}")
            return None
            
    elif 'bias' in name.lower():
        frame_type = 'bias'
        # Format: PIRATE_NUMBER_Bias22_ID_YYYY_MM_DD_HH_MM_SS
        try:
            frame_id = parts[3]
            date_parts = parts[4:7]  # YYYY, MM, DD
            date = '_'.join(date_parts)
            
            return {
                'type': frame_type,
                'filter': None,  # Bias frames don't have filters
                'date': date,
                'id': frame_id,
                'original': filename
            }
        except IndexError:
            print(f"Could not parse bias file: {filename}")
            return None
    
    else:
        print(f"Unknown file type: {filename}")
        return None


def create_new_filename(info):
    """
    Create simplified filename from parsed information.
    
    Parameters
    ----------
    info : dict
        Parsed file information
        
    Returns
    -------
    str
        New simplified filename
    """
    if info['type'] == 'flat':
        # Format: YYYY_MM_DD_FILTER_flat_ID.fits
        new_name = f"{info['date']}_{info['filter']}_flat_{info['id'].zfill(2)}.fits"
    else:  # bias
        # Format: YYYY_MM_DD_bias_ID.fits
        new_name = f"{info['date']}_bias_{info['id'].zfill(2)}.fits"
    
    return new_name

## Test Parsing on Sample Files

In [ ]:
# Test the parsing function
test_files = [
    'PIRATE_158371_flats_R_01_2025_09_01_19_54_25.fits',
    'PIRATE_161217_Bias22_0_2025_09_20_18_36_49.fits',
    'PIRATE_162424_flats_B_04_2025_09_25_19_17_25.fits',
    'PIRATE_163073_Bias22_1_2025_09_29_18_25_45.fits'
]

print("Testing filename parsing:")
print("=" * 70)
for filename in test_files:
    info = parse_filename(filename)
    if info:
        new_name = create_new_filename(info)
        print(f"Original: {filename}")
        print(f"New:      {new_name}")
        print(f"Type:     {info['type']}, Filter: {info['filter']}, Date: {info['date']}")
        print()

## Preview Reorganization (Dry Run)

In [ ]:
# Set paths
data_dir = Path('../data')
calibration_dir = data_dir / 'calibration'

print(f"Calibration directory: {calibration_dir.absolute()}")
print()

# Find all week directories
week_dirs = sorted([d for d in calibration_dir.glob('week*') if d.is_dir()])

if not week_dirs:
    print(f"No week directories found in {calibration_dir}")
else:
    print(f"Found {len(week_dirs)} week directories:")
    for week_dir in week_dirs:
        fits_count = len(list(week_dir.glob('*.fits')))
        print(f"  {week_dir.name}: {fits_count} FITS files")
    print()
    
    # Preview first few files from each week
    print("Sample files (first 2 from each week):")
    print("=" * 70)
    for week_dir in week_dirs:
        print(f"\n{week_dir.name}:")
        fits_files = sorted(week_dir.glob('*.fits'))[:2]
        for fits_file in fits_files:
            info = parse_filename(fits_file.name)
            if info:
                new_name = create_new_filename(info)
                print(f"  {fits_file.name}")
                print(f"  -> {info['type']}/{new_name}")

In [ ]:
def reorganize_calibration_files(calibration_dir, dry_run=True):
    """
    Reorganize calibration files from week folders into flats/ and bias/ folders.
    
    Parameters
    ----------
    calibration_dir : Path
        Path to calibration directory containing week1, week2, etc.
    dry_run : bool
        If True, only print what would be done without making changes
    """
    calibration_dir = Path(calibration_dir)
    
    # Create new directory structure
    flats_dir = calibration_dir / 'flats'
    bias_dir = calibration_dir / 'bias'
    
    if not dry_run:
        flats_dir.mkdir(exist_ok=True)
        bias_dir.mkdir(exist_ok=True)
        print(f"Created directories:")
        print(f"   {flats_dir}")
        print(f"   {bias_dir}")
    else:
        print(f"[DRY RUN] Would create directories:")
        print(f"   {flats_dir}")
        print(f"   {bias_dir}")
    
    print()
    
    # Find all week directories
    week_dirs = sorted([d for d in calibration_dir.glob('week*') if d.is_dir()])
    
    if not week_dirs:
        print(f"No week directories found in {calibration_dir}")
        return
    
    print(f"Found {len(week_dirs)} week directories: {[d.name for d in week_dirs]}\n")
    
    # Statistics
    stats = {
        'flats_moved': 0,
        'bias_moved': 0,
        'errors': 0,
        'duplicates': 0
    }
    
    # Track new filenames to detect duplicates
    new_filenames = {'flat': set(), 'bias': set()}
    
    # Process each week directory
    for week_dir in week_dirs:
        print(f"Processing {week_dir.name}...")
        
        # Find all FITS files
        fits_files = sorted(week_dir.glob('*.fits'))
        
        if not fits_files:
            print(f"  No FITS files found in {week_dir.name}")
            continue
        
        print(f"  Found {len(fits_files)} FITS files")
        
        for fits_file in fits_files:
            # Parse filename
            info = parse_filename(fits_file.name)
            
            if info is None:
                stats['errors'] += 1
                continue
            
            # Create new filename
            new_filename = create_new_filename(info)
            
            # Determine destination directory
            if info['type'] == 'flat':
                dest_dir = flats_dir
                file_type = 'flat'
            else:
                dest_dir = bias_dir
                file_type = 'bias'
            
            dest_path = dest_dir / new_filename
            
            # Check for duplicates
            if new_filename in new_filenames[file_type]:
                print(f" DUPLICATE: {new_filename}")
                # Add week number to filename to make it unique
                name_parts = new_filename.rsplit('.', 1)
                new_filename = f"{name_parts[0]}_{week_dir.name}.{name_parts[1]}"
                dest_path = dest_dir / new_filename
                stats['duplicates'] += 1
            
            new_filenames[file_type].add(new_filename)
            
            # Move or copy file
            if dry_run:
                if fits_files.index(fits_file) < 2:  # Only show first 2 per week
                    print(f"  [DRY RUN] {fits_file.name} -> {file_type}/{new_filename}")
            else:
                try:
                    shutil.copy2(fits_file, dest_path)
                    if info['type'] == 'flat':
                        stats['flats_moved'] += 1
                    else:
                        stats['bias_moved'] += 1
                except Exception as e:
                    print(f" Error copying {fits_file.name}: {e}")
                    stats['errors'] += 1
        
        print()
    
    # Print summary
    print("=" * 70)
    print("SUMMARY")
    print("=" * 70)
    if dry_run:
        print("[DRY RUN - No files were actually moved]")
    print(f"Flat fields: {stats['flats_moved']}")
    print(f"Bias frames: {stats['bias_moved']}")
    print(f"Duplicates handled: {stats['duplicates']}")
    print(f"Errors: {stats['errors']}")
    print()
    
    if not dry_run:
        print("✅ Reorganization complete!")
        print()
        print("Original week directories are preserved.")
        print("You can delete them manually if reorganization looks correct.")
    
    return stats

In [ ]:
# Run in DRY RUN mode first to preview
print("DRY RUN - Preview of reorganization")
print("=" * 70)
stats = reorganize_calibration_files(calibration_dir, dry_run=True)

In [ ]:
# UNCOMMENT AND RUN THIS CELL TO ACTUALLY REORGANIZE FILES
print("ACTUAL RUN - Reorganizing files")
print("=" * 70)
stats = reorganize_calibration_files(calibration_dir, dry_run=False)

## Verify New Structure

In [ ]:
# Check the new directory structure
flats_dir = calibration_dir / 'flats'
bias_dir = calibration_dir / 'bias'

if flats_dir.exists():
    flats = sorted(flats_dir.glob('*.fits'))
    print(f"Flats directory: {len(flats)} files")
    print("Sample files:")
    for f in flats[:5]:
        print(f"  {f.name}")
    if len(flats) > 5:
        print(f"  ... and {len(flats) - 5} more")
else:
    print("Flats directory not created yet")

print()

if bias_dir.exists():
    bias = sorted(bias_dir.glob('*.fits'))
    print(f"Bias directory: {len(bias)} files")
    print("Sample files:")
    for f in bias[:5]:
        print(f"  {f.name}")
    if len(bias) > 5:
        print(f"  ... and {len(bias) - 5} more")
else:
    print("Bias directory not created yet")

In [1]:
import os
from pathlib import Path

# Set the directory containing your files
standards_dir = Path("data/raw/standard_stars")

# Loop through all files in the directory
for file in standards_dir.glob("*star2*.fits"):
    new_name = file.name.replace("OLS", "OSL")
    new_path = file.parent / new_name
    os.rename(file, new_path)
    print(f"Renamed: {file.name} -> {new_name}")

Renamed: PIRATE_164643_OLS_ROE_EXO1_SA110_506_star2_00_SA110_506_00_Filter_B_02_2025_10_06_20_14_11.fits -> PIRATE_164643_OSL_ROE_EXO1_SA110_506_star2_00_SA110_506_00_Filter_B_02_2025_10_06_20_14_11.fits
Renamed: PIRATE_164645_OLS_ROE_EXO1_SA110_506_star2_00_SA110_506_00_Filter_I_02_2025_10_06_20_16_17.fits -> PIRATE_164645_OSL_ROE_EXO1_SA110_506_star2_00_SA110_506_00_Filter_I_02_2025_10_06_20_16_17.fits
Renamed: PIRATE_164635_OLS_ROE_EXO1_SA110_506_star2_00_SA110_506_00_Filter_R_00_2025_10_06_20_07_50.fits -> PIRATE_164635_OSL_ROE_EXO1_SA110_506_star2_00_SA110_506_00_Filter_R_00_2025_10_06_20_07_50.fits
Renamed: PIRATE_164632_OLS_ROE_EXO1_SA110_506_star2_00_SA110_506_00_Filter_I_00_2025_10_06_20_06_45.fits -> PIRATE_164632_OSL_ROE_EXO1_SA110_506_star2_00_SA110_506_00_Filter_I_00_2025_10_06_20_06_45.fits
Renamed: PIRATE_164637_OLS_ROE_EXO1_SA110_506_star2_00_SA110_506_00_Filter_V_00_2025_10_06_20_08_37.fits -> PIRATE_164637_OSL_ROE_EXO1_SA110_506_star2_00_SA110_506_00_Filter_V_00_2025_

In [ ]:
# Set the directory containing your files
standards_dir = Path("data/reduced/standard_stars")

# Loop through all files in the directory
for file in standards_dir.glob("*OSL*.fits"):
    new_path = file.parent / new_name
    os.rename(file, new_path)
    print(f"Renamed: {file.name} -> {new_name}")

In [4]:
# irectory containing reduced standard star files
STANDARDS_DIR = Path("data/reduced/standard_stars")

# Allowed PIRATE IDs (keep files that have these as the numeric token after "PIRATE_")
ALLOWED_IDS = {
    "164619","164618","164617",
    "164622","164621","164620",
    "164631","164630","164629",
    "164625","164624","164623",
    "164646","164645","164644",
    "164635","164636","164638",
    "164652","164651","164650",
    "164637","164639","164640",
    "164727","164728","164729",
    "164721","164720","164719",
    "164730","164731","164732",
    "164733","164734","164735",
    "164723","164722","164724",
    "164704","164702","164689",
    "164703","164705","164706",
    "164715","164714","164713",
    "164707","164708","164709",
    "164665","164664","164660",
    "164667","164668","164666",
    "164672","164674","164673",
    "164671","164669","164670",
}

# Safety options
DRY_RUN = False    # If True, only print what would be removed
MOVE_TO_REMOVED = False # If True and not dry-run, move removed files to "removed" folder instead of deleting

if not STANDARDS_DIR.exists():
    print(f"Directory not found: {STANDARDS_DIR.resolve()}")
    raise SystemExit(1)

removed_dir = STANDARDS_DIR / "removed_unwanted"
if MOVE_TO_REMOVED and not DRY_RUN:
    removed_dir.mkdir(exist_ok=True)

files = sorted(STANDARDS_DIR.glob("*.fits"))
print(f"Found {len(files)} .fits files in {STANDARDS_DIR}")

to_remove = []
to_keep = []

for f in files:
    name = f.name
    # Try to extract numeric ID after "PIRATE_"
    pirate_match = re.match(r"^PIRATE_(\d{6})_", name)
    if pirate_match:
        pid = pirate_match.group(1)
    else:
        # fallback: find first 6-digit sequence anywhere in the filename
        m = re.search(r"(\d{6})", name)
        pid = m.group(1) if m else None

    if pid and pid in ALLOWED_IDS:
        to_keep.append(f)
    else:
        to_remove.append((f, pid))

print(f"Will keep {len(to_keep)} files, will remove {len(to_remove)} files (DRY_RUN={DRY_RUN})")

# Show a short preview
if to_remove:
    print("\nSample files to remove:")
    for f, pid in to_remove[:20]:
        print(f"  {f.name}   (id={pid})")

# Perform removal / move
if not DRY_RUN:
    for f, pid in to_remove:
        try:
            if MOVE_TO_REMOVED:
                dest = removed_dir / f.name
                shutil.move(str(f), str(dest))
                print(f"Moved: {f.name} -> {removed_dir.name}/")
            else:
                f.unlink()
                print(f"Deleted: {f.name}")
        except Exception as e:
            print(f"Error handling {f.name}: {e}")
else:
    print("\nDRY RUN: no files were modified. Set DRY_RUN = False to actually remove/move files.")

Found 131 .fits files in data/reduced/standard_stars
Will keep 63 files, will remove 68 files (DRY_RUN=False)

Sample files to remove:
  PIRATE_164605_OSL_ROE_EXO1_SA111_1965_star1_00_SA111_1965_00_Filter_I_00_2025_10_06_19_34_41.fits   (id=164605)
  PIRATE_164606_OSL_ROE_EXO1_SA111_1965_star1_00_SA111_1965_00_Filter_R_00_2025_10_06_19_35_15.fits   (id=164606)
  PIRATE_164607_OSL_ROE_EXO1_SA111_1965_star1_00_SA111_1965_00_Filter_I_02_2025_10_06_19_35_42.fits   (id=164607)
  PIRATE_164608_OSL_ROE_EXO1_SA111_1965_star1_00_SA111_1965_00_Filter_I_01_2025_10_06_19_35_59.fits   (id=164608)
  PIRATE_164609_OSL_ROE_EXO1_SA111_1965_star1_00_SA111_1965_00_Filter_R_01_2025_10_06_19_36_29.fits   (id=164609)
  PIRATE_164610_OSL_ROE_EXO1_SA111_1965_star1_00_SA111_1965_00_Filter_V_00_2025_10_06_19_37_00.fits   (id=164610)
  PIRATE_164611_OSL_ROE_EXO1_SA111_1965_star1_00_SA111_1965_00_Filter_R_02_2025_10_06_19_37_27.fits   (id=164611)
  PIRATE_164612_OSL_ROE_EXO1_SA111_1965_star1_00_SA111_1965_00_Filt

In [3]:

from datetime import datetime

# Directory containing reduced standard star files
STANDARDS_DIR = Path("data/reduced/standard_stars")

# Safety options
DRY_RUN = False   # set False to perform renames
BACKUP = False    # if True, copy originals to backups/ before renaming

if not STANDARDS_DIR.exists():
    raise SystemExit(f"Directory not found: {STANDARDS_DIR.resolve()}")

# Regex to extract star number, filter band, and timestamp (if present)
# example name contains: ..._star1_..._Filter_I_00_2025_10_06_19_47_48.fits
pat = re.compile(r"star(\d+).*?Filter[_\-]?([A-Za-z]).*?(\d{4}_\d{2}_\d{2}_\d{2}_\d{2}_\d{2})", re.IGNORECASE)

groups = {}  # (star, filter) -> list of (Path, sort_key)

for p in sorted(STANDARDS_DIR.glob("*.fits")):
    m = pat.search(p.name)
    if not m:
        # skip files that don't match expected pattern
        print(f"SKIP  (no match): {p.name}")
        continue
    star = m.group(1)
    flt = m.group(2).upper()
    dt_token = m.group(3)
    try:
        sort_key = datetime.strptime(dt_token, "%Y_%m_%d_%H_%M_%S")
    except Exception:
        # fallback to mtime
        sort_key = datetime.fromtimestamp(p.stat().st_mtime)
    groups.setdefault((star, flt), []).append((p, sort_key))

# Prepare backup dir if needed
backup_dir = STANDARDS_DIR / "backup_before_rename"
if BACKUP and not DRY_RUN:
    backup_dir.mkdir(exist_ok=True)

# Perform renames
summary = []
for (star, flt), items in groups.items():
    # sort by timestamp (oldest first)
    items.sort(key=lambda x: x[1])
    for idx, (path, _) in enumerate(items, start=1):
        new_name = f"PIRATE_OSL_ROE_EXO1_star{star}_filter_{flt}_{idx}.fits"
        new_path = path.parent / new_name

        # avoid overwriting existing file
        if new_path.exists():
            # append a small suffix to avoid collision
            k = 1
            while True:
                candidate = path.parent / f"{new_path.stem}_{k}{new_path.suffix}"
                if not candidate.exists():
                    new_path = candidate
                    break
                k += 1

        summary.append((path.name, new_name))

        if DRY_RUN:
            print(f"[DRY] {path.name} -> {new_name}")
        else:
            if BACKUP:
                shutil.copy2(path, backup_dir / path.name)
            path.rename(new_path)
            print(f"RENAMED {path.name} -> {new_name}")

print()
print(f"Groups found: {len(groups)}")
for (star, flt), items in groups.items():
    print(f" star{star} filter {flt}: {len(items)} files")

print()
if DRY_RUN:
    print("DRY_RUN is True: no files were actually renamed. Set DRY_RUN = False to execute.")
else:
    print("Rename complete.")

SKIP  (no match): PIRATE_164660_OSL_ROE_EXO1_WASP135_00_WASP135_00_Filter_I_02_2025_10_06_20_29_33.fits
SKIP  (no match): PIRATE_164664_OSL_ROE_EXO1_WASP135_00_WASP135_00_Filter_I_01_2025_10_06_20_35_29.fits
SKIP  (no match): PIRATE_164665_OSL_ROE_EXO1_WASP135_00_WASP135_00_Filter_I_02_2025_10_06_20_37_01.fits
SKIP  (no match): PIRATE_164666_OSL_ROE_EXO1_WASP135_00_WASP135_00_Filter_R_01_2025_10_06_20_38_47.fits
SKIP  (no match): PIRATE_164667_OSL_ROE_EXO1_WASP135_00_WASP135_00_Filter_R_00_2025_10_06_20_40_19.fits
SKIP  (no match): PIRATE_164668_OSL_ROE_EXO1_WASP135_00_WASP135_00_Filter_R_02_2025_10_06_20_41_51.fits
SKIP  (no match): PIRATE_164669_OSL_ROE_EXO1_WASP135_00_WASP135_00_Filter_V_00_2025_10_06_20_43_34.fits
SKIP  (no match): PIRATE_164670_OSL_ROE_EXO1_WASP135_00_WASP135_00_Filter_V_01_2025_10_06_20_45_17.fits
SKIP  (no match): PIRATE_164671_OSL_ROE_EXO1_WASP135_00_WASP135_00_Filter_V_02_2025_10_06_20_46_59.fits
SKIP  (no match): PIRATE_164672_OSL_ROE_EXO1_WASP135_00_WASP135_

In [5]:
# ...existing code...
from datetime import datetime

# Directory containing reduced standard star files
STANDARDS_DIR = Path("data/reduced/standard_stars")

# Safety options
DRY_RUN = False   # set False to perform renames
BACKUP = False    # if True, copy originals to backup_before_rename/

if not STANDARDS_DIR.exists():
    raise SystemExit(f"Directory not found: {STANDARDS_DIR.resolve()}")

# regexes
obj_pat = re.compile(r"(WASP\d+)", re.IGNORECASE)                 # object token e.g. WASP135
flt_pat = re.compile(r"Filter[_\-]?([A-Za-z]+)", re.IGNORECASE)  # filter token e.g. B, V, I
time_pat = re.compile(r"(\d{4}_\d{2}_\d{2}_\d{2}_\d{2}_\d{2})")

# collect files per (object, filter) with timestamps for ordering
groups = {}  # (obj_upper, filter_upper) -> list of (Path, sort_key)

for p in sorted(STANDARDS_DIR.glob("*.fits")):
    name_up = p.name.upper()
    if "WASP" not in name_up:
        print(f"SKIP (not WASP): {p.name}")
        continue

    m_obj = obj_pat.search(p.name)
    m_flt = flt_pat.search(p.name)
    if not m_obj or not m_flt:
        print(f"SKIP (no object/filter): {p.name}")
        continue

    obj = m_obj.group(1).upper()
    flt = m_flt.group(1).upper()

    m_time = time_pat.search(p.name)
    if m_time:
        try:
            sort_key = datetime.strptime(m_time.group(1), "%Y_%m_%d_%H_%M_%S")
        except Exception:
            sort_key = datetime.fromtimestamp(p.stat().st_mtime)
    else:
        sort_key = datetime.fromtimestamp(p.stat().st_mtime)

    groups.setdefault((obj, flt), []).append((p, sort_key))

# prepare backup dir if requested
backup_dir = STANDARDS_DIR / "backup_before_rename"
if BACKUP and not DRY_RUN:
    backup_dir.mkdir(exist_ok=True)

# perform renames: index per (object, filter) ordered by timestamp (oldest first)
summary = []
for (obj, flt), items in groups.items():
    items.sort(key=lambda x: x[1])
    for idx, (path, _) in enumerate(items, start=1):
        new_name = f"PIRATE_OSL_ROE_EXO1_{obj}_Filter_{flt}_{idx}.fits"
        new_path = path.parent / new_name

        # avoid overwriting existing file
        if new_path.exists():
            k = 1
            while True:
                candidate = path.parent / f"{new_path.stem}_{k}{new_path.suffix}"
                if not candidate.exists():
                    new_path = candidate
                    break
                k += 1

        summary.append((path.name, new_name))

        if DRY_RUN:
            print(f"[DRY] {path.name} -> {new_name}")
        else:
            if BACKUP:
                shutil.copy2(path, backup_dir / path.name)
            path.rename(new_path)
            print(f"RENAMED {path.name} -> {new_name}")

print()
print(f"WASP groups found: {len(groups)}")
for (obj, flt), items in groups.items():
    print(f" {obj} Filter {flt}: {len(items)} files")

print()
if DRY_RUN:
    print("DRY_RUN is True: no files were actually renamed. Set DRY_RUN = False to execute.")
else:
    print("Rename complete.")
# ...existing code...

SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star1_filter_B_1.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star1_filter_B_2.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star1_filter_B_3.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star1_filter_I_1.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star1_filter_I_2.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star1_filter_I_3.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star1_filter_R_1.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star1_filter_R_2.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star1_filter_R_3.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star1_filter_V_1.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star1_filter_V_2.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star1_filter_V_3.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star2_filter_B_1.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star2_filter_B_2.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star2_filter_B_3.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star2_filter_I_1.fits
SKIP (not WASP): PIRATE_OSL_ROE_EXO1_star2_filter_I_2.fi